In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
import os
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# 경고 무시 모듈
import warnings

# 경고를 무시
warnings.filterwarnings(action='ignore')

In [32]:
encoding= 'cp949'
file_name = 'agg_pcell_time_service_pop_2022_1'
User_point = pd.read_csv(f'./전처리/grid/{file_name}.csv', encoding=encoding)

In [33]:
User_point

gid  recordid                                           geom  \
0      30728485  37655633   POINT (1123733.991787065 1711829.1473899658)   
1      30766272  37689642  POINT (1123233.9917870644 1711529.1473899654)   
2      30766273  37689643  POINT (1123283.9917870667 1711529.1473899658)   
3      30766276  37689646   POINT (1123433.991787067 1711529.1473899654)   
4      30772589  37695327    POINT (1123233.991787064 1711479.147389966)   
...         ...       ...                                            ...   
74492  34327160  40894440   POINT (1125383.991787071 1684879.1473899633)   
74493  34333930  40900532  POINT (1124933.9917870678 1684829.1473899635)   
74494  34333931  40900533    POINT (1124983.991787067 1684829.147389964)   
74495  34333932  40900534   POINT (1125033.991787069 1684829.1473899637)   
74496  34333933  40900535   POINT (1125083.991787069 1684829.1473899642)   

              hcode    year  month  h_pop  w_pop  v_pop  sum_pop  
0      4.825035e+09  2022.0    1.0    7.0    2.0    0.0     10.0  
1      4.825035e+09  2022.0    1.0    0.0    0.0    0.0      0.0  
2      4.825035e+09  2022.0    1.0    0.0    0.0    0.0      0.0  
3      4.825035e+09  2022.0    1.0    0.0    0.0    0.0      0.0  
4      4.825035e+09  2022.0    1.0    0.0    0.0    0.0      0.0  
...             ...     ...    ...    ...    ...    ...      ...  
74492  4.825063e+09  2022.0    1.0  896.0   35.0   91.0   1022.0  
74493  4.825063e+09  2022.0    1.0   17.0    0.0    0.0     17.0  
74494  4.825063e+09  2022.0    1.0   14.0    0.0    0.0     14.0  
74495  4.825063e+09  2022.0    1.0   14.0    0.0    0.0     14.0  
74496  4.825063e+09  2022.0    1.0   14.0    0.0    0.0     14.0  

[74497 rows x 10 columns]

In [4]:
city_recycle = gpd.GeoDataFrame.from_file('./원본데이터/전통시장_유동인구_구역/전통시장_유동인구_구역.shp', 
                                          encoding=encoding, crs=epsg4326)

In [5]:
city_recycle

id       전통시장명                                           geometry
0   1      진영전통시장  POLYGON ((128.72923 35.30203, 128.72926 35.303...
1   2      장유전통시장  POLYGON ((128.81399 35.20015, 128.81401 35.201...
2   3      외동전통시장  POLYGON ((128.86532 35.23225, 128.86575 35.234...
3   4  김해(동상)전통시장  POLYGON ((128.88203 35.23508, 128.88207 35.236...
4   5      삼방전통시장  POLYGON ((128.90905 35.24898, 128.90854 35.250...

In [6]:
encoding= 'cp949'
file_name = 'agg_pcell_time_service_pop_2022_1~12'



epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
epsg5179 = from_string("+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs")




# 도시재생화사업지
city_recycle = gpd.GeoDataFrame.from_file('./원본데이터/전통시장_유동인구_구역/전통시장_유동인구_구역.shp', 
                                          encoding=encoding, crs=epsg4326)
city_recycle =  city_recycle.to_crs(epsg5179)

city_recycle.rename(columns={'전통시장명':'name'}, inplace=True)


# 그리드와 병합할 포인트
User_point = pd.read_csv(f'./전처리/grid/{file_name}.csv', encoding=encoding)

User_point['x'] = None
User_point['y'] = None

for n in tqdm(range(len(User_point))):
    xy = User_point['geom'][n].split()
    User_point['x'][n] = float(xy[1].split('(')[1])
    User_point['y'][n] = float(xy[2].split(')')[0])

User_point.drop('geom', axis=1, inplace=True)
User_point['geometry'] = User_point.apply(lambda row : Point([row['x'], row['y']]), axis=1)

User_point = gpd.GeoDataFrame(User_point, geometry='geometry', crs = epsg5179)


# 공간 병합
result = gpd.sjoin(city_recycle, User_point, how='left', op="intersects")

result = result.groupby(['id','name'])['h_pop','w_pop','v_pop','sum_pop'].sum()
result.reset_index(inplace=True)

  0%|          | 0/82749 [00:00<?, ?it/s]

In [8]:
result = result.astype({
           'h_pop': int, 'w_pop': int,
           'v_pop':int, 'sum_pop':int})

In [10]:
result.drop(['h_pop','w_pop','sum_pop'], axis=1,inplace=True)
result.rename(columns={'v_pop':'pop'}, inplace=True)

In [11]:
result['Month_mean'] = round(result['pop']/12)

In [13]:
result = result.astype({'Month_mean':int})

In [14]:
result

id        name      pop  Month_mean
0   1      진영전통시장  1165832       97153
1   2      장유전통시장  1229483      102457
2   3      외동전통시장  3093652      257804
3   4  김해(동상)전통시장  2705181      225432
4   5      삼방전통시장   689921       57493

In [15]:
result.to_excel('./전처리/전통시장/전통시장_연합계_월평균.xlsx')

In [20]:
def city_recycle(file_name, encoding='cp949'):
    epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    epsg5179 = from_string("+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs")

    # 도시재생화사업지
    city_recycle = gpd.GeoDataFrame.from_file('./원본데이터/전통시장_유동인구_구역/전통시장_유동인구_구역.shp', 
                                              encoding=encoding, crs=epsg4326)
    city_recycle =  city_recycle.to_crs(epsg5179)
    
    city_recycle.rename(columns={'전통시장명':'name'}, inplace=True)
    
    # 그리드와 병합할 포인트
    User_point = pd.read_csv(f'./전처리/grid/{file_name}.csv', encoding=encoding)

    User_point['x'] = None
    User_point['y'] = None

    for n in tqdm(range(len(User_point))):
        xy = User_point['geom'][n].split()
        User_point['x'][n] = float(xy[1].split('(')[1])
        User_point['y'][n] = float(xy[2].split(')')[0])

    User_point.drop('geom', axis=1, inplace=True)
    User_point['geometry'] = User_point.apply(lambda row : Point([row['x'], row['y']]), axis=1)

    User_point = gpd.GeoDataFrame(User_point, geometry='geometry', crs = epsg5179)

    # 공간 병합
    result = gpd.sjoin(city_recycle, User_point, how='left', op="intersects")
    
    result = result.groupby(['id','name', 'month'])['h_pop','w_pop','v_pop','sum_pop'].sum()
    result.reset_index(inplace=True)
    
    result = result.astype({'month':int,
               'h_pop': int, 'w_pop': int,
               'v_pop':int, 'sum_pop':int})
    
    result.sort_values('id', inplace=True)
    result.reset_index(drop=True, inplace=True)
    city_recycle.sort_values('id', inplace=True)
    city_recycle.reset_index(drop=True, inplace=True)
    city_recycle_geo = city_recycle[['geometry']]
    
    df = pd.concat([result, city_recycle_geo], axis=1)
    
    month = df['month'][0]
    
    # GeoPandas로 변경
    df = gpd.GeoDataFrame(df, geometry='geometry', crs = epsg5179)
    
    # 폴더 생성
    try:
        directory = f'./전처리/전통시장/csv/'
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        
    
    # csv파일 저장
    df.to_csv(f'{directory}{file_name}.csv', encoding='cp949',index=False)

In [21]:
for n in tqdm(range(1,13)):
    city_recycle(f'agg_pcell_time_service_pop_2022_{n}')

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/74497 [00:00<?, ?it/s]

  0%|          | 0/74800 [00:00<?, ?it/s]

  0%|          | 0/74465 [00:00<?, ?it/s]

  0%|          | 0/75963 [00:00<?, ?it/s]

...

  0%|          | 0/77208 [00:00<?, ?it/s]

  0%|          | 0/77181 [00:00<?, ?it/s]

  0%|          | 0/77381 [00:00<?, ?it/s]

  0%|          | 0/77537 [00:00<?, ?it/s]

  0%|          | 0/75485 [00:00<?, ?it/s]

In [29]:
for n in range(1,13):
    df = pd.read_csv(f'./전처리/전통시장/csv/agg_pcell_time_service_pop_2022_{n}.csv', encoding='cp949')
    
    if n == 1:
        cc_df = df
        continue
    cc_df = pd.concat([cc_df, df])

In [30]:
cc_df.drop(['geometry'],axis=1,inplace=True)

In [31]:
cc_df.sort_values(['id','month'],inplace=True)

In [32]:
cc_df.reset_index(drop=True, inplace=True)

In [33]:
cc_df.columns

Index(['id', 'name', 'month', 'h_pop', 'w_pop', 'v_pop', 'sum_pop'], dtype='object')

In [34]:
cc_df.set_index(['id','name','month'], inplace=True)

In [35]:
cc_df

h_pop   w_pop   v_pop  sum_pop
id name       month                                  
1  진영전통시장     1       760842   75407  100076   936361
              2       687068   65048   86339   838486
              3       652599   66135   74187   792952
              4       660187   73134   83865   817215
              5       664558   74993   93165   832738
              6       668852   73251   86453   828589
              7       772753   83427   99696   955897
              8       772463   76254  103692   952441
              9       744868   78854  105719   929466
              10      831685   89537  114488  1035736
              11      804862   89283  106480  1000656
              12      852222   87397  111513  1051159
2  장유전통시장     1       620889   48513  113987   783393
              2       561329   41464   96015   698811
              3       556661   44478   88955   690098
              4       493719   44508   90471   628703
              5       494516   44717   98528   637766
              6       483084   42240   94926   620254
              7       458061   39676  107527   605269
              8       442310   36027  107153   585494
              9       463409   40464  109779   613658
              10      540319   47278  104614   692216
              11      542986   49021   97001   689011
              12      672177   58109  120502   850792
3  외동전통시장     1      2433318  172378  280332  2886038
              2      2151009  143999  233486  2528502
              3      2528071  199180  257574  2984836
              4      2454014  207549  277281  2938853
              5      2481873  211976  301984  2995840
              6      2461306  210433  289243  2960990
              7      1965996  154335  237222  2357562
              8      2022601  156493  245134  2424240
              9      1989079  158410  250782  2398285
              10     1947536  155786  229029  2332357
              11     1873581  159181  218336  2251111
              12     2150014  175777  273197  2598995
4  김해(동상)전통시장 1      1088403   96384  281633  1466442
              2       968825   89616  233208  1291664
              3       881728   94354  199676  1175780
              4       821847   86828  204064  1112762
              5       833017   89556  233795  1156384
              6       818037   86312  216576  1120945
              7       861683   74108  230318  1166133
              8       856033   70306  225671  1152032
              9       775286   75913  214213  1065431
              10      843399   77997  224010  1145428
              11      820126   85492  212880  1118518
              12      874176   84024  229035  1187257
5  삼방전통시장     1       484962   43390   61612   589975
              2       431003   36570   54099   521684
              3       494852   50436   60639   605941
              4       570810   61557   80905   713282
              5       441541   46339   64472   552361
              6       442217   45956   57742   545923
              7       429821   42045   52321   524197
              8       406359   36313   50915   493599
              9       406766   40639   56884   504297
              10      404546   40004   55380   499940
              11      385755   41120   51168   478049
              12      344436   33009   43728   421181

In [36]:
cc_df.to_excel('./전처리/전통시장/전통시장_월별.xlsx')

In [1]:
import platform
from matplotlib import font_manager, rc # Runtime Configuration

path = "c:/Windows/Fonts/malgun.ttf"
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')

In [6]:
df = pd.read_excel('./전처리/전통시장/전통시장_월별.xlsx')

In [7]:
df

id        name  month     pop
0    1      진영전통시장      1  100076
1    1      진영전통시장      2   86339
2    1      진영전통시장      3   74187
3    1      진영전통시장      4   83865
4    1      진영전통시장      5   93165
5    1      진영전통시장      6   86453
6    1      진영전통시장      7   99696
7    1      진영전통시장      8  103692
8    1      진영전통시장      9  105719
9    1      진영전통시장     10  114488
10   1      진영전통시장     11  106480
11   1      진영전통시장     12  111513
12   2      장유전통시장      1  113987
13   2      장유전통시장      2   96015
14   2      장유전통시장      3   88955
15   2      장유전통시장      4   90471
16   2      장유전통시장      5   98528
17   2      장유전통시장      6   94926
18   2      장유전통시장      7  107527
19   2      장유전통시장      8  107153
20   2      장유전통시장      9  109779
21   2      장유전통시장     10  104614
22   2      장유전통시장     11   97001
23   2      장유전통시장     12  120502
24   3      외동전통시장      1  280332
25   3      외동전통시장      2  233486
26   3      외동전통시장      3  257574
27   3      외동전통시장      4  277281
28   3      외동전통시장      5  301984
29   3      외동전통시장      6  289243
30   3      외동전통시장      7  237222
31   3      외동전통시장      8  245134
32   3      외동전통시장      9  250782
33   3      외동전통시장     10  229029
34   3      외동전통시장     11  218336
35   3      외동전통시장     12  273197
36   4  김해(동상)전통시장      1  281633
37   4  김해(동상)전통시장      2  233208
38   4  김해(동상)전통시장      3  199676
39   4  김해(동상)전통시장      4  204064
40   4  김해(동상)전통시장      5  233795
41   4  김해(동상)전통시장      6  216576
42   4  김해(동상)전통시장      7  230318
43   4  김해(동상)전통시장      8  225671
44   4  김해(동상)전통시장      9  214213
45   4  김해(동상)전통시장     10  224010
46   4  김해(동상)전통시장     11  212880
47   4  김해(동상)전통시장     12  229035
48   5      삼방전통시장      1   61612
49   5      삼방전통시장      2   54099
50   5      삼방전통시장      3   60639
51   5      삼방전통시장      4   80905
52   5      삼방전통시장      5   64472
53   5      삼방전통시장      6   57742
54   5      삼방전통시장      7   52321
55   5      삼방전통시장      8   50915
56   5      삼방전통시장      9   56884
57   5      삼방전통시장     10   55380
58   5      삼방전통시장     11   51168
59   5      삼방전통시장     12   43728

In [8]:
pdf1 = pd.pivot_table(df,                
                     index = 'month',    # 행 위치에 들어갈 열
                     columns = 'name',    # 열 위치에 들어갈 열
                     values = ['pop']     # 데이터로 사용할 열
                     )
pdf1

pop                               
name  김해(동상)전통시장 삼방전통시장  외동전통시장  장유전통시장  진영전통시장
month                                          
1         281633  61612  280332  113987  100076
2         233208  54099  233486   96015   86339
3         199676  60639  257574   88955   74187
4         204064  80905  277281   90471   83865
5         233795  64472  301984   98528   93165
6         216576  57742  289243   94926   86453
7         230318  52321  237222  107527   99696
8         225671  50915  245134  107153  103692
9         214213  56884  250782  109779  105719
10        224010  55380  229029  104614  114488
11        212880  51168  218336   97001  106480
12        229035  43728  273197  120502  111513

In [31]:
pdf1.to_excel('./전처리/전통시장/전통시장_피벗.xlsx')

In [9]:
pdf1.columns

MultiIndex([('pop', '김해(동상)전통시장'),
            ('pop',     '삼방전통시장'),
            ('pop',     '외동전통시장'),
            ('pop',     '장유전통시장'),
            ('pop',     '진영전통시장')],
           names=[None, 'name'])

In [30]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(15, 10), facecolor='w')
fig.subplots_adjust(hspace = 0.07)
bar_width = 0.2

plt.rc('font', size=15)

x = []
for n in pdf1.index:
    x.append(f'{n}월')

y_1 = pdf1[('pop', '김해(동상)전통시장')]
y_2 = pdf1[('pop', '삼방전통시장')]
y_3 = pdf1[('pop', '외동전통시장')]
y_4 = pdf1[('pop', '장유전통시장')]
y_5 = pdf1[('pop', '진영전통시장')]


ax2.plot(x, y_2, linewidth='3',
        color='r',marker='o',markersize=15)
ax1.plot(x, y_3, linewidth='3',
        color='#f19400',marker='o',markersize=15)
ax2.plot(x, y_4, linewidth='3',
        color='#80b700',marker='o',markersize=15)
ax2.plot(x, y_5, linewidth='3',
        color='#4d4d4d',marker='o',markersize=15)

ax1.plot(x, y_1, linewidth='3',
        color='#0058ef',marker='o',markersize=15)

        
ax1.set_ylim(190000, 350000)
ax2.set_ylim(0, 160000)

ax1.set_axisbelow(True)
ax1.grid(color='gray', linestyle='dashed')

ax2.set_axisbelow(True)
ax2.grid(color='gray', linestyle='dashed')

plt.xlabel('월별', labelpad=20, size=20, weight ='bold')
ax1.set_ylabel('유동인구', fontsize=20,labelpad=20, weight ='bold')

plt.subplots_adjust(wspace=0, hspace=0.1)


current_values_1 = ax1.get_yticks()
ax1.set_yticklabels(['{:,.0f}'.format(x) for x in current_values_1])

current_values_2 = ax2.get_yticks()
ax2.set_yticklabels(['{:,.0f}'.format(x) for x in current_values_2])


ax1.legend(['외동전통시장','김해(동상)전통시장'], loc = (0.7, 0.77))
ax2.legend(['삼방전통시장',
            '장유전통시장', '진영전통시장'], loc = (0.7, 1.58))

plt.show()